# Applications

## Tagging - POS

**Parts of speed tagging** - Gán nhãn loại từ:

- *POS Tagging* là quá trình gán nhãn (label) loại từ cho mỗi từ trong một câu, ví dụ như danh từ (N), động từ (V), tính từ (ADJ), trạng từ (ADV), giới từ (PREP), liên từ (CONJ), v.v.
- *Mục tiêu*: Từ một chuỗi các từ, dự đoán lớp từ tương ứng sao cho phản ánh đúng ngữ pháp và ngữ nghĩa trong ngôn ngữ đó.

**Ý nghĩa**

- Xác định loại từ là bước cơ bản và cần thiết trong nhiều ứng dụng NLP phức tạp hơn, ví dụ:
    - *Parsing*: Xây dựng cây phân tích cú pháp (syntactic parsing).
    - *Dịch máy*: Hỗ trợ xác định cấu trúc ngữ pháp, lựa chọn từ tương ứng chính xác.
    - *Trích xuất thông tin*: Hỗ trợ nhận diện và phân loại từ, cụm từ đặc biệt.
- POS Tagging giúp máy tính hiểu được chức năng ngữ pháp của từ trong câu, từ đó xử lý ngôn ngữ hiệu quả hơn.

**Phương pháp**

- *Thống kê cổ điển (HMM, CRF)*: Các mô hình ẩn Markov (HMM) và Mô hình trường ngẫu nhiên có điều kiện (CRF) dựa vào xác suất chuyển tiếp giữa các nhãn và xác suất quan sát từ để dự đoán nhãn POS.
- *Học sâu (Deep Learning)*: Các mô hình mạng nơ-ron (RNN, LSTM, BiLSTM, Transformers) kết hợp với embeddings (như Word2Vec, GloVe, hay contextual embeddings như BERT) cũng có thể được huấn luyện để gán nhãn POS một cách chính xác hơn, nhất là với những ngôn ngữ giàu ngữ cảnh như tiếng Việt.

In [2]:
import spacy
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt

data = pd.read_csv(
    r"contents\theory\aiml_algorithms\dl_nlp\data\94 - bbc-news.csv"
)
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Preprocessing
titles = data[["title"]].copy()

# lower case
titles["preprocessed"] = titles["title"].str.lower()

# stop word removal
en_stopwords = stopwords.words("english")
titles["preprocessed"] = titles["preprocessed"].apply(
    lambda x: " ".join(
        [word for word in x.split() if word not in en_stopwords]
    )
)

# punctation removal
titles["preprocessed"] = titles["preprocessed"].apply(
    lambda x: re.sub(r"[^\w\s]", "", x)
)

# word tokenization
titles["tokens_raw"] = titles["title"].apply(word_tokenize)
titles["tokens_preprocessed"] = titles["preprocessed"].apply(word_tokenize)

# lemmatizing
lemmatizer = WordNetLemmatizer()
titles["tokens_lemmatized"] = titles["tokens_preprocessed"].apply(
    lambda x: [lemmatizer.lemmatize(word) for word in x]
)

# create lists for just our tokens
tokens_raw_list = sum(
    titles["tokens_raw"], []
)  # unpack our lists into a single list
tokens_clean_list = sum(titles["tokens_lemmatized"], [])

In [5]:
# create a spacy doc from our raw text - better for pos tagging
spacy_doc = nlp(" ".join(tokens_raw_list))

In [ ]:
# POS Tagging
pos = [{"token": token.text, "pos_tag": token.pos_} for token in spacy_doc]
pd.DataFrame(pos).head()

,token,pos_tag
0,Can,AUX
1,I,PRON
2,refuse,VERB
3,to,PART
4,work,VERB


## Tagging - NER

**Named Entity Recognition** – Nhận diện thực thể có tên

- *NER* là quá trình tự động phát hiện (dò tìm) và phân loại các thực thể “có tên” trong văn bản, ví dụ như tên người, tên địa điểm, tổ chức, thời gian, sự kiện, v.v.
- *Mục tiêu*: Xác định đâu là thực thể đặc biệt trong câu và chúng thuộc loại thực thể nào.
```text
Ví dụ câu tiếng Việt: “Ông Nguyễn Văn A làm việc tại Công ty Bkav ở Hà Nội.”
“Nguyễn Văn A” → Người (Person)
“Công ty Bkav” → Tổ chức (Organization)
“Hà Nội” → Địa điểm (Location)
```

**Ý nghĩa** 

NER là một bước quan trọng trong trích xuất thông tin:

- *Tìm kiếm và tổng hợp thông tin*: Cho phép hệ thống hiểu được đối tượng nào đang được đề cập đến (ví dụ, tìm tất cả các tin tức có nhắc đến một người/tổ chức cụ thể).
- *Hỏi đáp tự động (Question Answering)*: Giúp xác định câu trả lời chính xác khi câu hỏi hướng đến các thực thể (Ví dụ: “Ai là CEO của Google?”).
- *Phân tích ý kiến (Sentiment Analysis)*: Xem ý kiến của người dùng nhắm vào thực thể nào (nhãn hàng, địa điểm, cá nhân).

**Phương pháp**

- *Rule-based*: Sử dụng luật ngôn ngữ hoặc biểu thức chính quy (regex), kết hợp danh sách các thực thể đã biết. Cách này mang tính đặc thù, thiếu linh hoạt.
- *Thống kê/học máy cổ điển*: CRF, MaxEnt (Maximum Entropy), SVM… sử dụng đặc trưng (feature) về từ, ngữ cảnh, chữ hoa/thường, tiền tố/hậu tố, v.v.
- *Học sâu (Deep Learning)*:
    - Mô hình mạng nơ-ron nhiều tầng (BiLSTM + CRF) hoặc Transformers (BERT, XLM-R, PhoBERT…) huấn luyện đặc trưng tự động, thay vì thủ công.
    - Kết quả NER hiện nay cải thiện rất nhiều nhờ áp dụng mô hình Transformer, đặc biệt khi có dữ liệu gắn nhãn đủ lớn.

In [ ]:
ner = [
    {"token": token.text, "ner_tag": token.label_} for token in spacy_doc.ents
]
df = pd.DataFrame(ner).head()
df

,token,ner_tag
0,Liz Truss,PERSON
1,UK,GPE
2,Rationing,PRODUCT
3,superyachts,CARDINAL
4,Russian,NORP


In [ ]:
from spacy import displacy

displacy.render(spacy_doc[:150], style="ent", jupyter=True)

In [19]:
type(spacy_doc)

spacy.tokens.doc.Doc